In [85]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.saving import register_keras_serializable

In [86]:
california_housing = fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 

# Splitting data into train and test.

In [87]:
X_train0, X_test, Y_train0, Y_test  = train_test_split(california_housing["data"], california_housing["target"])

In [88]:
X_train1, X_validation, Y_train1, Y_validation = train_test_split(X_train0, Y_train0)

In [89]:
X_test.shape

(5160, 8)

In [90]:
X_train0.shape

(15480, 8)

In [91]:
X_train1.shape

(11610, 8)

In [92]:
X_validation.shape

(3870, 8)

# Normalizing data
using "sklearn.preprocessing.StandardScaler" function for normalizing data

In [93]:
sc = StandardScaler()
X_train_s = sc.fit_transform(X_train1)
X_validation_s = sc.fit_transform(X_validation)
X_test_s = sc.transform(X_test)

--------------------------------

In [94]:
X_train0.shape[1:]

(8,)

In [95]:
X_train_s_1, X_train_s_2 = X_train_s[:, :6], X_train_s[:, :-4]
X_validation_s_1, X_validation_s_2 = X_validation_s[:, :6], X_validation_s[:, :-4]
X_test_s_1, X_test_s_2 = X_test_s[:, :6], X_test_s[:, :-4]

# Subclass API

In [96]:
@register_keras_serializable()
class WideAndDeepANN(keras.Model):
    def __init__(self, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer_1 = keras.layers.Dense(50, activation=activation)
        self.hidden_layer_2 = keras.layers.Dense(10, activation=activation)
        self.final_output = keras.layers.Dense(1)
        self.helper_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_1, input_2 = inputs
        h1_out = self.hidden_layer_1(input_1)
        h2_out = self.hidden_layer_2(h1_out)
        concat_out = keras.layers.concatenate([input_2, h2_out])
        final_out = self.final_output(concat_out)
        helper_out = self.helper_output(h2_out)
        return final_out, helper_out

In [97]:
model_f = WideAndDeepANN()

# Compile Model

In [98]:
model_f.compile(loss = ["mse", "mse"],
              loss_weights = [0.8, 0.2],
              optimizer = "sgd",
              metrics = [["mean_absolute_error"], ["mean_absolute_error"]])

# Fit Model

In [99]:
model_f.fit((X_train_s_1, X_train_s_2),(Y_train1, Y_train1), epochs=30, validation_data = ((X_validation_s_1, X_validation_s_2), (Y_validation, Y_validation)))

Epoch 1/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.6421 - mean_absolute_error: 0.8510 - mean_absolute_error_1: 1.1745 - mse_loss: 2.5214 - val_loss: 0.6468 - val_mean_absolute_error: 0.5684 - val_mean_absolute_error_1: 0.6399 - val_mse_loss: 0.7057
Epoch 2/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6003 - mean_absolute_error: 0.5565 - mean_absolute_error_1: 0.6127 - mse_loss: 0.6547 - val_loss: 0.5762 - val_mean_absolute_error: 0.5543 - val_mean_absolute_error_1: 0.5857 - val_mse_loss: 0.6016
Epoch 3/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5820 - mean_absolute_error: 0.5542 - mean_absolute_error_1: 0.5688 - mse_loss: 0.5935 - val_loss: 0.5600 - val_mean_absolute_error: 0.5559 - val_mean_absolute_error_1: 0.5875 - val_mse_loss: 0.5913
Epoch 4/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5600 - mean_absolute_error: 0.5511 - mean_absolute_error_1: 0.5568 - mse_loss: 0.5685 - val_loss: 0.6697 - val_mean_absolute_error: 0.5839 - val_mean_absolute

In [100]:
model_f.save("housing_reg_model.keras")

In [101]:
model_f_reg = keras.models.load_model("housing_reg_model.keras")